In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [6]:
metadata_dementia = pd.read_excel("./data/sea-ad_cohort_donor_metadata_072524 (1).xlsx")
markers = pd.read_excel("./data/sea-ad_cohort_mtg-tissue_extractions-luminex_data (1).xlsx", header=1)
markers_metadata = markers.merge(metadata_dementia, on="Donor ID", how="inner")

In [16]:
# We have boolean variable (has_dementia) and continuous variable (the markers) so that's why
# we use the Point-biserial correlation test

has_dementia = markers_metadata["Cognitive Status"] == "Dementia"
stat_tests = []

stat_tests.append(("ABeta40 pg/ug", stats.pointbiserialr(has_dementia, markers_metadata["ABeta40 pg/ug"])))
stat_tests.append(("ABeta40 pg/ug.1", stats.pointbiserialr(has_dementia, markers_metadata["ABeta40 pg/ug.1"])))

stat_tests.append(("ABeta42 pg/ug", stats.pointbiserialr(has_dementia, markers_metadata["ABeta42 pg/ug"])))
stat_tests.append(("ABeta42 pg/ug.1", stats.pointbiserialr(has_dementia, markers_metadata["ABeta42 pg/ug.1"])))


stat_tests.append(("tTAU pg/ug", stats.pointbiserialr(has_dementia, markers_metadata["tTAU pg/ug"])))
stat_tests.append(("tTAU pg/ug.1", stats.pointbiserialr(has_dementia, markers_metadata["tTAU pg/ug.1"])))


stat_tests.append(("pTAU pg/ug", stats.pointbiserialr(has_dementia, markers_metadata["pTAU pg/ug"])))
stat_tests.append(("pTAU pg/ug.1", stats.pointbiserialr(has_dementia, markers_metadata["pTAU pg/ug.1"])))

stat_tests.append(("pTAU pg/ug", stats.pointbiserialr(has_dementia, markers_metadata["pTAU pg/ug"])))
stat_tests.append(("pTAU pg/ug.1", stats.pointbiserialr(has_dementia, markers_metadata["pTAU pg/ug.1"])))

stat_tests.sort(key=lambda x: x[1].pvalue)
for marker_name, stat_result in stat_tests:
    print(marker_name, stat_result)

ABeta42 pg/ug.1 SignificanceResult(statistic=np.float64(0.36926431479639504), pvalue=np.float64(0.0005468860229021458))
ABeta40 pg/ug SignificanceResult(statistic=np.float64(0.1923433925773067), pvalue=np.float64(0.07962795819019367))
ABeta42 pg/ug SignificanceResult(statistic=np.float64(0.18929282995609398), pvalue=np.float64(0.08461213592817061))
ABeta40 pg/ug.1 SignificanceResult(statistic=np.float64(0.18605310633004082), pvalue=np.float64(0.09017336911961892))
pTAU pg/ug SignificanceResult(statistic=np.float64(0.17517290353561218), pvalue=np.float64(0.11098536353592969))
pTAU pg/ug SignificanceResult(statistic=np.float64(0.17517290353561218), pvalue=np.float64(0.11098536353592969))
tTAU pg/ug.1 SignificanceResult(statistic=np.float64(-0.16526498871761805), pvalue=np.float64(0.13301082174880974))
tTAU pg/ug SignificanceResult(statistic=np.float64(-0.12209989238947337), pvalue=np.float64(0.26853526734965566))
pTAU pg/ug.1 SignificanceResult(statistic=np.float64(-0.06602811051858709),